<a href="https://colab.research.google.com/github/AnaBelenCarbajal/Thesis/blob/main/shapesTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osculari

In [64]:
# importing required packages
import osculari

import numpy as np
from matplotlib import pyplot as plt
import torch
import os

from torchvision.datasets import ImageFolder

In [ ]:
# unzip folder (previously uploaded in collab)
!unzip images180.zip -d my_data

In [92]:
images180 = "my_data/images180/images180/"

In [90]:
labels = os.listdir("my_data/images180/images180/")

In [89]:
images + labels[0]

'my_data/images180/images180/leopard'

In [ ]:
data = []

for label in labels:
  if label == ".DS_Store":
    continue
  folder_path = images180 + label
  shapes = os.listdir(folder_path)
  for shape in shapes:
    category_dict = {'label': label, 'image': shape}
    data.append(category_dict)

data

In [117]:
def image_direct(root, category_dict_item):
  return root + category_dict_item['label'] + "/" + category_dict_item['image']

In [107]:
folder_path

'my_data/images180/images180/rat'

In [85]:
[{label: os.listdir(path + label)} for label in labels if label != ".DS_Store"]

[{'leopard': ['41.png',
   '22.png',
   '89.png',
   '46.png',
   '31.png',
   '69.png',
   '98.png',
   '1.png',
   '39.png',
   '24.png',
   '19.png',
   '44.png',
   '37.png',
   '97.png',
   '74.png',
   '35.png',
   '32.png',
   '78.png',
   '10.png',
   '7.png',
   '99.png',
   '82.png',
   '16.png',
   '14.png',
   '83.png',
   '70.png',
   '18.png',
   '11.png',
   '45.png',
   '81.png',
   '85.png',
   '63.png',
   '13.png',
   '26.png',
   '91.png',
   '57.png',
   '49.png',
   '20.png',
   '5.png',
   '68.png',
   '59.png',
   '15.png',
   '84.png',
   '64.png',
   '80.png',
   '27.png',
   '92.png',
   '73.png',
   '76.png',
   '79.png',
   '86.png',
   '48.png',
   '95.png',
   '55.png',
   '53.png',
   '34.png',
   '42.png',
   '67.png',
   '93.png',
   '9.png',
   '17.png',
   '87.png',
   '65.png',
   '100.png',
   '66.png',
   '28.png',
   '61.png',
   '71.png',
   '90.png',
   '75.png',
   '6.png',
   '47.png',
   '72.png',
   '29.png',
   '60.png',
   '38.png',
   '6

In [67]:
labels[7]

'elephant'

In [48]:
!ls my_data/images180/images180

bird	   cat	crocodile  dog	     duck      fish	   hen	  leopard  rabbit  spider
butterfly  cow	deer	   dolphine  elephant  flyingbird  horse  monkey   rat	   tortoise


In [77]:
a = ImageFolder("my_data/images180/images180")


(<PIL.Image.Image image mode=RGB size=180x180>, 0)

In [74]:
from torch.utils.data import DataLoader

In [75]:
b = DataLoader(a, 64)